In [29]:
import pandas as pd

print pd.__version__




'''
import pickle


kosdaq = pickle.load(open('./kosdaq.pickle')) 
kospi = pickle.load(open('./kospi.pickle')) 


import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np


style.use('ggplot')


start = dt.datetime(2013, 1, 1)
end = dt.datetime(2016, 12, 31)

df = web.DataReader('AAPL', "yahoo", start, end)

#print(df.head())

keep_col = ['Open', 'High', 'Low', 'Volume', 'Close'] # except 'Date', 'Adj Close'

new_df = df[keep_col][:]

new_df.to_csv("inputs.csv", index=False, header=False)


print "Shape: ", np.array(new_df).shape

'''


0.22.0


'\nimport pickle\n\n\nkosdaq = pickle.load(open(\'./kosdaq.pickle\')) \nkospi = pickle.load(open(\'./kospi.pickle\')) \n\n\nimport datetime as dt\nimport matplotlib.pyplot as plt\nfrom matplotlib import style\nimport pandas as pd\nimport pandas_datareader.data as web\nimport numpy as np\n\n\nstyle.use(\'ggplot\')\n\n\nstart = dt.datetime(2013, 1, 1)\nend = dt.datetime(2016, 12, 31)\n\ndf = web.DataReader(\'AAPL\', "yahoo", start, end)\n\n#print(df.head())\n\nkeep_col = [\'Open\', \'High\', \'Low\', \'Volume\', \'Close\'] # except \'Date\', \'Adj Close\'\n\nnew_df = df[keep_col][:]\n\nnew_df.to_csv("inputs.csv", index=False, header=False)\n\n\nprint "Shape: ", np.array(new_df).shape\n\n'